# 1.1.1.1: Definición y Representación de Vectores

## Objetivos de Aprendizaje

Al completar este notebook, serás capaz de:

- **Explicar** qué es un vector, mencionando sus dos componentes clave: magnitud y dirección.
- **Diferenciar** entre la representación numérica (componentes) y gráfica (flecha) de un vector.
- **Definir** qué es la dimensión de un vector y usar la notación $ℝ^n$ correctamente.
- **Visualizar** vectores en 2D y 3D usando SageMath y Python, aplicados a diversos contextos de datos.
- **Conectar** el concepto abstracto de vector con su aplicación en ciencia de datos (features).

In [ ]:
# --- Celda de Configuración (Oculta) ---
# Ejecuta esta celda una vez para configurar el notebook.

# 1. Renderizado matemático con LaTeX para mayor claridad.
%display latex

# 2. Importaciones de librerías estándar de Python.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import os

# 3. Función de ayuda para graficar vectores 2D de forma consistente en SageMath.
def plot_vector_2d(vectors, colors=None, legends=None, title="Gráfico de Vectores 2D"):
    """Función para graficar una lista de vectores 2D con una paleta accesible."""
    if colors is None:
        # Paleta de colores accesible (Okabe-Ito)
        colors = ['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7']
    
    plots = []
    for i, v in enumerate(vectors):
        p_legend = legends[i] if legends and i < len(legends) else str(v)
        p_color = colors[i % len(colors)]
        plots.append(plot(v, color=p_color, legend_label=p_legend, thickness=1.5))
    
    combined_plot = sum(plots)
    combined_plot.show(aspect_ratio=1, gridlines=True, title=title)

## 1. Idea Central y Aplicación en Ciencia de Datos

Un **vector** es un objeto matemático que empaqueta dos piezas de información: **magnitud** (¿cuánto?) y **dirección** (¿hacia dónde?). Es la forma de dar una instrucción de desplazamiento precisa desde un origen común.

> **Por qué importa en Ciencia de Datos:** Piensa en un vector como una **fila de tu dataset** o una **observación**. Si tienes datos de un cliente, el vector `[edad, ingresos, compras_mes]` lo posiciona en un "espacio de clientes". La matemática de vectores nos permite medir distancias y similitudes entre estos puntos, que es la base de algoritmos como k-NN, k-Means y PCA.

## 2. Representación, Dimensión y Notación $ℝ^n$

La **dimensión** de un vector es simplemente el número de componentes (o *features*) que tiene. Usamos la notación **$ℝ^n$** para describir el "espacio" de alta dimensión en el que vive un vector de dimensión *n*.

| Vector | Dimensión | Espacio | Significado en Data Science |
| :--- | :--- | :--- | :--- |
| `[3, 4]` | 2 | $ℝ^2$ | Una observación con 2 features (ej: `[horas_estudio, nota]`). |
| `[5, -1, 3]` | 3 | $ℝ^3$ | Una observación con 3 features (ej: `[precio, marketing, ventas]`). |
| `[c₁, c₂, ..., cₙ]` | n | $ℝ^n$ | Una observación con *n* features. ¡El pan de cada día! |

--- 
## ⚙️ El Arsenal de Datasets: Nuestra Fuente de Ejercicios

Para que nuestro aprendizaje sea coherente y aplicado, usaremos un **arsenal de funciones para generar datos**. Este enfoque es superior a usar archivos estáticos porque nos permite crear una variedad infinita de escenarios y controlar las propiedades de los datos para cada ejercicio. 

En la siguiente celda, configuraremos todo lo necesario. Importaremos las funciones de generación que usaremos en este notebook y crearemos una instancia central de un generador de números aleatorios para asegurar que nuestros resultados sean **reproducibles**.

In [ ]:
# === CONFIGURACIÓN DE DATASETS ===
import numpy as np
import pandas as pd

# 1. Importar las funciones de generación de datos que usaremos en este notebook
# Nota: Solo importamos las que son relevantes para el tema de 'Vectores'.
from src.data_generation.create_student_performance import create_student_performance_data
from src.data_generation.create_geometric_shapes import create_geometric_shapes
from src.data_generation.create_business_data import create_business_data
from src.data_generation.create_edge_cases import create_edge_cases

# 2. Configuración centralizada de aleatoriedad para REPRODUCIBILIDAD
# Todas las funciones de generación usarán este mismo 'rng'.
rng = np.random.default_rng(seed=42)

# 3. Generación de los datasets principales que usaremos en ejemplos y ejercicios
# 💡 CONTEXTO PEDAGÓGICO: Hilo Conductor
# Usamos datos de rendimiento estudiantil (simplificado a 2D) como nuestro ejemplo principal.
# La relación lineal entre 'horas_estudio' y 'calificacion_examen' es fácil de visualizar
# e ideal para introducir conceptos vectoriales de forma intuitiva.
datos_estudiantes_simplificado = create_student_performance_data(rng, simplified=True, n_samples=100)

# 💡 CONTEXTO PEDAGÓGICO: Intuición Geométrica
# Usamos formas geométricas puras para quitar el "ruido" de un contexto real y centrarnos
# exclusivamente en la geometría de los vectores: su posición, dirección y magnitud.
datos_circulo = create_geometric_shapes(rng, shape_type="circle", n_samples=40, noise_level=0.05)
datos_linea = create_geometric_shapes(rng, shape_type="line", n_samples=20, noise_level=0.2)

# 💡 CONTEXTO PEDAGÓGICO: Aplicación a Negocios
# Este dataset nos permite enmarcar los vectores en un problema de negocio, donde cada
# vector puede representar un producto con un 'precio' y un volumen de 'ventas'.
datos_negocio = create_business_data(rng, simplified=True, n_samples=75)

print("Datasets generados y listos para usar:")
print(f"- Hilo Conductor (Estudiantes): {datos_estudiantes_simplificado.shape}")
print(f"- Geometría (Círculo): {datos_circulo.shape}")
print(f"- Geometría (Línea): {datos_linea.shape}")
print(f"- Negocios (Productos): {datos_negocio.shape}")

datos_estudiantes_simplificado.head()

--- 
## 3. Ejemplos Demostrativos (5+)

Antes de los ejercicios, veamos varios ejemplos resueltos y comentados que cubren diferentes contextos y establecen los patrones de código que usaremos.

### Ejemplo Demostrativo 1: Visualizar un Único Vector en $ℝ^2$

In [ ]:
# 1. Definimos el vector v en el espacio R^2
v = vector([4, 3])

# 2. Creamos el objeto gráfico y lo personalizamos
grafico_v = plot(v, color='#D55E00', legend_label='v = (4, 3)', thickness=2)

# 3. Mostramos el gráfico con opciones de estilo para mejor visualización
# aspect_ratio=1 asegura que los ejes tengan la misma escala, evitando distorsiones.
grafico_v.show(aspect_ratio=1, gridlines=True, title="Ejemplo 1: Vector Único")

### Ejemplo Demostrativo 2: Múltiples Vectores con la Función de Ayuda

In [ ]:
# 1. Definimos una lista de vectores
vec_a = vector([-3, 1])
vec_b = vector([1, -2])
vec_c = vector([4, 2])

# 2. Usamos nuestra función de ayuda para graficarlos todos a la vez
# La función se encarga de asignar colores y leyendas de forma automática.
plot_vector_2d(
    [vec_a, vec_b, vec_c], 
    legends=['a = (-3, 1)', 'b = (1, -2)', 'c = (4, 2)'],
    title="Ejemplo 2: Múltiples Vectores"
)

### Ejemplo Demostrativo 3: Visualizar un Vector 3D Interactivo

In [ ]:
# 1. Definimos un vector en R^3
u = vector([3, 4, 5])

# 2. Creamos el gráfico 3D
grafico_3d = plot(u, color='green', thickness=4)

# 3. Usamos el visor 'threejs' para que sea interactivo
# ¡Puedes rotar, hacer pan y zoom en el gráfico resultante con el mouse!
grafico_3d.show(viewer='threejs', title="Ejemplo 3: Vector 3D Interactivo")

### Ejemplo Demostrativo 4: Vectores del Hilo Conductor (Estudiantes)

In [ ]:
# 1. Extraemos los datos de dos estudiantes de nuestro DataFrame
# Usamos .values para obtener un array de NumPy, que SageMath puede convertir a vector.
estudiante_1_data = datos_estudiantes_simplificado.iloc[0].values
estudiante_alto_rendimiento_data = datos_estudiantes_simplificado.iloc[90].values # Un estudiante con buenos resultados

# 2. Creamos los objetos vector de SageMath
vec_estudiante_1 = vector(estudiante_1_data)
vec_estudiante_alto = vector(estudiante_alto_rendimiento_data)

# 3. INTERPRETACIÓN PEDAGÓGICA
# Visualizamos los vectores. Cada vector representa a un estudiante en el espacio de
# (horas de estudio, calificación). La dirección y longitud del vector nos dan una idea
# rápida de su perfil de rendimiento.
print(f"Vector del Estudiante 1 (Horas, Calif): {vec_estudiante_1}")
print(f"Vector de Alto Rendimiento (Horas, Calif): {vec_estudiante_alto}")

# 4. VISUALIZACIÓN
plot_vector_2d(
    [vec_estudiante_1, vec_estudiante_alto],
    legends=['Estudiante 1', 'Estudiante de Alto Rendimiento'],
    title="Ejemplo 4: Vectores del Dataset de Estudiantes"
)

### Ejemplo Demostrativo 5: Vectores en un Contexto Geométrico (Círculo)

In [ ]:
# 1. GENERACIÓN DE DATOS (Ya la hicimos en la celda de config)
# Usamos 'datos_circulo'

# 2. APLICACIÓN DEL CONCEPTO
# Tomamos tres puntos del círculo y los convertimos en vectores desde el origen.
puntos_circulo = datos_circulo.sample(n=3, random_state=1) # Tomamos 3 puntos al azar
vectores_circulo = [vector(row) for row in puntos_circulo.values]

# 3. INTERPRETACIÓN PEDAGÓGICA
# En un contexto puramente geométrico, los vectores nos ayudan a definir la posición
# de puntos en el espacio. Observa cómo todos los vectores tienen una magnitud (longitud)
# muy similar, porque todos yacen en la circunferencia de un círculo.
print("Tres vectores extraídos de la forma de círculo:")
for v in vectores_circulo:
    print(f"- Vector: {v}, Magnitud: {v.norm():.2f}")

# 4. VISUALIZACIÓN
# Primero graficamos todos los puntos del círculo como referencia
puntos_plot = points(datos_circulo.values, color='gray', alpha=0.5, size=10)
# Luego, superponemos nuestros vectores
vectores_plot = plot_vector_2d(vectores_circulo, title="Ejemplo 5: Vectores en un Círculo")

(puntos_plot + vectores_plot).show(aspect_ratio=1, gridlines=True)

### Ejemplo Demostrativo 6: Vectores en un Contexto de Negocios

In [ ]:
# 1. DATOS PROPORCIONADOS (Ya generados)
# Usamos 'datos_negocio', que contiene 'precio' y 'ventas_mensuales'

# 2. APLICACIÓN DEL CONCEPTO
# Buscamos dos productos con perfiles opuestos para compararlos.
producto_barato_exitoso = datos_negocio.sort_values(by='ventas_mensuales', ascending=False).iloc[0]
producto_caro_nicho = datos_negocio.sort_values(by='precio', ascending=False).iloc[0]

vec_exito = vector(producto_barato_exitoso.values)
vec_nicho = vector(producto_caro_nicho.values)

# 3. INTERPRETACIÓN PEDAGÓGICA
# Cada vector representa un producto en el 'espacio de productos'.
# El vector 'Éxito en Ventas' apunta hacia pocas ventas y bajo precio.
# El vector 'Producto de Nicho' apunta hacia un alto precio pero menos ventas.
# Esta representación vectorial nos permitirá en el futuro encontrar 'productos similares'.
print(f"Vector Éxito (Precio, Ventas): {vec_exito}")
print(f"Vector Nicho (Precio, Ventas): {vec_nicho}")

# 4. VISUALIZACIÓN
plot_vector_2d(
    [vec_exito, vec_nicho],
    legends=['Éxito en Ventas', 'Producto de Nicho'],
    title="Ejemplo 6: Vectores de Productos en el Mercado"
)

---## 4. Ejercicios Guiados con Scaffolding (8+)
Estos ejercicios te guían paso a paso para asegurar que comprendes el proceso. Rellena las partes marcadas con `# COMPLETAR`.

In [ ]:
# === EJERCICIO GUIADO 1: Extraer y Visualizar un Vector de Estudiante ===

# DATOS PROPORCIONADOS
# Usaremos 'datos_estudiantes_simplificado'.
print("Dataset de Estudiantes (primeras 5 filas):")
display(datos_estudiantes_simplificado.head())

# TODO 1: Extrae los datos del estudiante en el índice 10.
# PISTA: Usa .iloc[10].values para obtener los valores como un array de NumPy.
datos_estudiante_10 = # COMPLETAR

# TODO 2: Convierte los datos del estudiante en un objeto vector de SageMath.
# PISTA: Usa la función vector().
vector_estudiante_10 = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
assert isinstance(vector_estudiante_10, sage.modules.vector_real_double.Vector_real_double_dense), "El resultado no es un vector de SageMath!"
print(f"\n✅ ¡Correcto! Vector del estudiante 10: {vector_estudiante_10}")

# TODO 3: Usa la función plot_vector_2d para visualizar tu vector.
# PISTA: Debe estar dentro de una lista, ej: [mi_vector].
# COMPLETAR

In [ ]:
# === EJERCICIO GUIADO 2: Calcular la Magnitud de un Vector de Estudiante ===

# DATOS PROPORCIONADOS
vector_estudiante_10 = vector(datos_estudiantes_simplificado.iloc[10].values)

# TODO 1: Calcula la magnitud (norma) del vector 'vector_estudiante_10'.
# PISTA: Los vectores de SageMath tienen un método .norm().
magnitud = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
expected_magnitude = np.linalg.norm(datos_estudiantes_simplificado.iloc[10].values)
assert np.isclose(magnitud, expected_magnitude), "La magnitud no es correcta."
print(f"✅ ¡Correcto! La magnitud del vector es: {magnitud:.2f}")
print("Interpretación: Este número representa la 'distancia' euclidiana del estudiante al origen (0,0).")

In [ ]:
# === EJERCICIO GUIADO 3: Extraer Vectores de una Forma Geométrica ===

# DATOS PROPORCIONADOS
# Usaremos 'datos_linea'.
print("Dataset Geométrico (Línea):")
display(datos_linea.head())

# TODO 1: Extrae el primer punto (índice 0) y el último punto (índice -1) del DataFrame.
# PISTA: Usa .iloc[...].values
primer_punto = # COMPLETAR
ultimo_punto = # COMPLETAR

# TODO 2: Convierte ambos puntos en vectores de SageMath.
vec_primero = # COMPLETAR
vec_ultimo = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
assert vec_primero.degree() == 2 and vec_ultimo.degree() == 2, "Los objetos no son vectores 2D."
print("\n✅ ¡Correcto! Vectores extraídos:")
print(f"- Primer vector: {vec_primero}")
print(f"- Último vector: {vec_ultimo}")

# TODO 3: Grafica AMBOS vectores en el mismo plano usando plot_vector_2d.
# No olvides pasar una lista de leyendas, como ['Primer Punto', 'Último Punto'].
# COMPLETAR

In [ ]:
# === EJERCICIO GUIADO 4: Identificar un Vector de Producto por Característica ===

# DATOS PROPORCIONADOS
# Usaremos 'datos_negocio'.

# TODO 1: Encuentra la fila correspondiente al producto con el precio más bajo.
# PISTA: Usa .sort_values(by='...') y luego .iloc[0].
producto_mas_barato = # COMPLETAR

# TODO 2: Convierte esta fila en un vector de SageMath.
vec_barato = vector(producto_mas_barato.values)

# VERIFICACIÓN AUTOMÁTICA
min_price = datos_negocio['precio'].min()
assert np.isclose(vec_barato[0], min_price), "No has seleccionado el producto correcto."
print(f"✅ ¡Correcto! El producto más barato tiene el vector (Precio, Ventas): {vec_barato}")

# TODO 3: ¿Qué esperas de la componente 'ventas_mensuales' de este vector? ¿Será alta o baja?
# Escribe tu respuesta en un comentario.

# Respuesta: # COMPLETAR

# Visualiza el vector para confirmar tu hipótesis.
plot_vector_2d([vec_barato], legends=['Producto más barato'])

In [ ]:
# === EJERCICIO GUIADO 5: Construir un Vector 3D ===

# DATOS: Usaremos la versión completa del dataset de estudiantes.
rng_temp = np.random.default_rng(123)
datos_estudiantes_3d = create_student_performance_data(rng_temp, simplified=False, n_samples=10)[['horas_estudio', 'calificacion_previa', 'calificacion_examen']]
print("Dataset 3D de Estudiantes:")
display(datos_estudiantes_3d.head())

# TODO 1: Extrae los datos del estudiante en el índice 3 en un vector de SageMath.
vec_estudiante_3d = # COMPLETAR

# TODO 2: Calcula la magnitud del vector 3D.
magnitud_3d = # COMPLETAR

print(f"\n✅ Vector 3D: {vec_estudiante_3d}")
print(f"✅ Magnitud: {magnitud_3d:.2f}")

# TODO 3: Crea un gráfico 3D interactivo para visualizar el vector.
# PISTA: Usa plot() y .show(viewer='threejs').
# COMPLETAR

In [ ]:
# === EJERCICIO GUIADO 6: El Vector Nulo ===

# TODO 1: Crea un vector nulo (cero) de dimensión 2.
vector_nulo_2d = # COMPLETAR

# TODO 2: Crea un vector nulo de dimensión 3.
vector_nulo_3d = # COMPLETAR

# TODO 3: Calcula la magnitud de ambos vectores.
mag_2d = # COMPLETAR
mag_3d = # COMPLETAR

# VERIFICACIÓN
assert mag_2d == 0 and mag_3d == 0, "La magnitud del vector nulo debe ser 0."
print(f"✅ ¡Correcto! La magnitud de {vector_nulo_2d} es {mag_2d}.")
print("El vector nulo tiene magnitud cero y una dirección indefinida. Es simplemente el origen.")

# ¿Qué pasa si intentas graficarlo? Descomenta la línea de abajo y observa.
# plot(vector_nulo_2d).show()

In [ ]:
# === EJERCICIO GUIADO 7: Vector como Punto en el Espacio ===

# Un vector puede verse como una flecha o como el punto final de esa flecha.
# Vamos a visualizar ambas ideas a la vez.

# TODO 1: Define un vector 'v' con componentes [5, 2].
v = # COMPLETAR

# TODO 2: Crea un gráfico del vector 'v' (una flecha).
plot_flecha = # COMPLETAR

# TODO 3: Crea un gráfico de un punto en las coordenadas (5, 2).
# PISTA: Usa points([(5, 2)], color='red', size=40).
plot_punto = # COMPLETAR

# Visualización combinada
print("Visualizando un vector como flecha y como punto final.")
(plot_flecha + plot_punto).show(aspect_ratio=1, gridlines=True)

In [ ]:
# === EJERCICIO GUIADO 8: Identificando Outliers Visualmente ===

# DATOS PROPORCIONADOS
# Generamos datos con algunos outliers (valores atípicos).
datos_con_outliers = create_edge_cases(rng, case_type="outliers", n_samples=50)

# TODO 1: Convierte CADA fila del DataFrame en un vector de SageMath.
# PISTA: Usa una comprensión de lista: [vector(row) for row in dataframe.values]
vectores_outliers = # COMPLETAR

# TODO 2: Grafica todos los vectores usando plot_vector_2d.
plot_all = # COMPLETAR

# TODO 3: En un comentario, describe qué hace que los vectores outliers se vean diferentes.
# ¿Es su magnitud, su dirección, o ambas cosas?

# Respuesta: # COMPLETAR

# VERIFICACIÓN VISUAL
plot_all.show()

--- 
# 5. Banco de Ejercicios Prácticos (30+)
Ahora te toca a ti. Resuelve estos ejercicios para consolidar tu conocimiento. Están organizados por dificultad y contexto.

### Parte A: Fundamentos de Visualización 2D (Hilo Conductor)

**A1 (🟢 Fácil):** Grafica el vector $\vec{v} = [4, 2]$ usando `plot()` y `.show()`.

**A2 (🟢 Fácil):** Grafica el vector $\vec{u} = [-1, 3]$ con la función de ayuda `plot_vector_2d`.

**A3 (🟢 Fácil):** Grafica los vectores $\vec{a} = [2, 5]$ y $\vec{b} = [5, 2]$ en el mismo plano. ¿Son iguales? ¿Por qué?

**A4 (🟢 Fácil):** Extrae el vector del estudiante en el índice 25 de `datos_estudiantes_simplificado` y visualízalo.

**A5 (🟡 Medio):** Grafica el vector $\vec{p} = [2, 1]$ y su opuesto, $-\vec{p}$. ¿Qué relación geométrica observas en términos de dirección y magnitud?

**A6 (🟡 Medio):** Grafica los vectores de los estudiantes en los índices 5, 50 y 95 de `datos_estudiantes_simplificado` en el mismo gráfico. Usa leyendas para identificarlos.

**A7 (🟡 Medio):** Encuentra al estudiante con el mayor número de horas de estudio en `datos_estudiantes_simplificado`. Crea un vector para este estudiante y visualízalo.

**A8 (🔴 Reto):** Crea un scatter plot de todos los puntos en `datos_estudiantes_simplificado` usando `points()`. Luego, superpón los vectores de los estudiantes con la calificación más alta y más baja. ¿Confirma la visualización la relación esperada?

### Parte B: Intuición con Formas Geométricas

**B1 (🟢 Fácil):** Extrae dos vectores cualesquiera del dataset `datos_circulo` y visualízalos. Comprueba si sus magnitudes son similares.

**B2 (🟢 Fácil):** Extrae dos vectores cualesquiera del dataset `datos_linea` y visualízalos. ¿Apuntan en una dirección similar?

**B3 (🟡 Medio):** Genera un nuevo dataset de una parábola (`shape_type='parabola'`). Extrae 3 puntos de la parábola (uno cerca del origen, dos más lejanos) y grafica sus vectores. ¿Cómo cambia la "pendiente" de los vectores a medida que te alejas del origen?

**B4 (🟡 Medio):** Un **vector libre** no parte del origen. Va de un punto A a un punto B, y se calcula como $B-A$. Usando `datos_circulo`, elige dos puntos A y B, calcula el vector libre $\vec{v} = B - A$ y visualízalo. Para visualizarlo desde el origen, simplemente usa `plot(v)`.

**B5 (🔴 Reto):** Grafica un cuadrado en el plano 2D cuyos vértices son (1,1), (3,1), (3,3) y (1,3). Para ello, grafica los cuatro vectores que van desde el origen a cada uno de estos vértices.

### Parte C: Aplicaciones en Contexto de Negocios

**C1 (🟢 Fácil):** Del dataset `datos_negocio`, extrae y visualiza el vector del producto en el índice 50.

**C2 (🟡 Medio):** Encuentra el producto con el mayor número de ventas en `datos_negocio`. Crea un vector para él y visualízalo. Compara su dirección con el del producto más caro (que probablemente tendrá pocas ventas).

**C3 (🟡 Medio):** Calcula la magnitud promedio de todos los vectores de productos en `datos_negocio`.

**C4 (🔴 Reto):** Filtra `datos_negocio` para quedarte solo con los productos cuyo precio es mayor a 250. De este subconjunto, extrae y visualiza 3 vectores al azar. ¿En qué cuadrante del plano tienden a agruparse estos vectores?

### Parte D: Exploración en 3D

**D1 (🟢 Fácil):** Grafica el vector 3D $\vec{p} = [1, 2, 3]$ con el visor interactivo.

**D2 (🟢 Fácil):** Grafica el vector 3D $\vec{q} = [-2, 3, -1]$ con el visor interactivo.

**D3 (🟡 Medio):** Grafica los vectores base canónicos en 3D: $\vec{i} = [1, 0, 0]$, $\vec{j} = [0, 1, 0]$ y $\vec{k} = [0, 0, 1]$ en el mismo gráfico 3D. Usa `sum()` para combinar los plots.

**D4 (🟡 Medio):** Genera un dataset de estudiantes completo (no simplificado) con 3 variables: `['horas_estudio', 'calificacion_previa', 'asistencia_porcentaje']`. Extrae al primer estudiante como un vector 3D y visualízalo.

### Parte E: Práctica con Python/Matplotlib

**E1 (🟢 Fácil):** Replica el ejercicio A1, graficando $\vec{v} = [4, 2]$ con Matplotlib.

In [ ]:
# Template para Matplotlib
fig, ax = plt.subplots(figsize=(5,5))
v = np.array([4, 2])
# Usa ax.quiver para dibujar la flecha desde el origen (0,0)
ax.quiver(0, 0, v[0], v[1], angles='xy', scale_units='xy', scale=1, color='#009E73')
ax.set_xlim(0, 5); ax.set_ylim(0, 5)
ax.set_title("Vector con Matplotlib")
ax.set_aspect('equal'); ax.grid(True); plt.show()

**E2 (🟡 Medio):** Replica el ejercicio A3, graficando $\vec{a} = [2, 5]$ y $\vec{b} = [5, 2]$ con Matplotlib en el mismo gráfico, con diferentes colores y una leyenda.

**E3 (🔴 Reto):** Crea un scatter plot del `datos_negocio` usando Matplotlib (`ax.scatter`). Luego, superpón (con `ax.quiver`) los vectores del producto más caro y el más barato. Esto combina un gráfico de datos con la visualización de vectores específicos.

### Parte F: Desafíos Conceptuales y de Dimensión

**F1 (Conceptual):** ¿Cuál es la dimensión del vector nulo $\vec{z} = [0, 0, 0]$? ¿Tiene magnitud? ¿Tiene dirección?

**F2 (Conceptual):** Si tomas un vector del dataset `datos_estudiantes_simplificado`, ¿en qué espacio ($ℝ^n$) vive? ¿Y si tomas un vector del dataset completo de estudiantes con 5 variables?

**F3 (🔴 Reto):** Crea tu propio "vector personal" de dimensión 4. Por ejemplo: `mi_vector = [edad, altura_cm, horas_sueño_promedio, tazas_cafe_diarias]`.
1. Define el vector en SageMath.
2. Calcula su magnitud (norma).
3. Explica qué representa esta magnitud en tu contexto personal (es un número abstracto, ¡pero intenta darle un significado!).
4. No puedes graficar un vector 4D, pero puedes graficar una proyección 2D. Grafica el vector `[edad, altura_cm]`.

**F4 (🔴 Reto):** Vuelve a generar el `datos_estudiantes` pero en su versión completa (`simplified=False`) y con 200 muestras. Luego, selecciona únicamente las columnas `['calificacion_previa', 'asistencia_porcentaje', 'tutor_privado', 'calificacion_examen']`. Esto te dará un DataFrame con datos en $ℝ^4$. Verifica sus dimensiones con `.shape`. No necesitas graficarlo, solo comprueba que has creado con éxito vectores de 4 dimensiones.

---

## ✅ Mini-Quiz de Autoevaluación

*Responde estas preguntas para verificar tu comprensión.*

1. ¿Cuál es la dimensión del vector `[2, -1, 0, 5]` y en qué espacio vive?
2. Si dos vectores tienen los mismos componentes pero en diferente orden (ej. `[1, 4]` y `[4, 1]`), ¿son el mismo vector? ¿Por qué?
3. Verdadero o Falso: La magnitud de un vector puede ser negativa.
4. ¿Qué representa un vector en el contexto de un dataset de Pandas?
5. ¿Qué método usarías en un vector de SageMath para calcular su longitud?

## 🚀 Próximos Pasos

¡Felicidades por completar esta introducción! Has visualizado vectores en múltiples contextos y, más importante, has aprendido a generar, cargar y representar datos de diversas fuentes, un flujo de trabajo esencial.

En el notebook **`1.1.1.2_Operaciones_Vectoriales.ipynb`**, aprenderás a manipular estos vectores. Descubrirás cómo sumar los datos de dos estudiantes, cómo "escalar" su rendimiento y por qué estas operaciones son la base de casi todo en machine learning. También analizaremos la eficiencia de estas operaciones (¡prepárate para la **vectorización**!).